In [1]:
# /usr/bin/python3

__author__ = "Martin Cepeda"
import os
import numpy as np
import plotly.graph_objs as go

import dash
import dash_core_components as dcc
import dash_bootstrap_components as dbc
import dash_html_components as html
from dash.dependencies import Input, Output

In [2]:
def makegrid(xMax, yMax, mu, sigma, nu, resolution=1000):
    
    assert xMax > 0
    assert yMax > 0
    assert nu > 0
    
    mu1, mu2 = mu
    sigma1, sigma2 = sigma
    
    nx = 2*resolution*int(xMax)
    ny = resolution*int(yMax)
    
    x_dom = np.linspace(-xMax, xMax, nx)
    y_dom = np.linspace(1e-5, yMax, ny)
    
    xx, yy = np.meshgrid(x_dom, y_dom, sparse=False, indexing='ij')
    zz = np.zeros((nx, ny))
    
    for i in range(nx):
        for j in range(ny):
            x = xx[i, j]
            y = yy[i, j]
            if y < nu:
                zz[i, j] = np.exp(-sigma1**2*(x - mu1)**2 - sigma2**2*(np.log(y/nu)*nu + nu- mu2)**2)
            else:
                zz[i, j] = np.exp(-sigma1**2*(x - mu1)**2 - sigma2**2*(y - mu2)**2)
    
    return xx, yy, zz

In [13]:
external_stylesheets = ['https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css',
                        'https://cdnjs.cloudflare.com/ajax/libs/highlight.js/9.18.1/styles/monokai-sublime.min.css']

external_scripts = ['https://code.jquery.com/jquery-3.2.1.slim.min.js',
                    'https://cdnjs.cloudflare.com/ajax/libs/popper.js/1.12.9/umd/popper.min.js',
                    'https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/js/bootstrap.min.js',]

filepath = 'C:\\Users\\marti\\Desktop\\Untitled Folder\\dash_app'
text_top = open(os.path.join(filepath, "text_top.md"), "r").read()
text_bottom = open(os.path.join(filepath, "text_bottom.md"), "r").read()

app = dash.Dash(__name__,
                external_stylesheets=external_stylesheets,
                external_scripts=external_scripts)
## Components

def display_plot(data):
    xx, yy, zz = data
    fig = go.Figure(data=[go.Surface(z=zz, x=xx, y=yy)])
    fig.update_layout(autosize=False, width=800, height=600, template='simple_white',
                      scene_camera_eye=dict(x=-1.5, y=-1.5, z=-0.3))
    fig.update_traces(contours_z=dict(show=True, usecolormap=True, highlightcolor="limegreen", project_z=True))
    return fig

## Sliders

slider_x_max  = dcc.Slider(id='x_max', min=1e-5, max=10, step=0.1, value=5.0, 
                           tooltip={'always_visible':True},updatemode='drag')
slider_y_max  = dcc.Slider(id='y_max', min=1e-5, max=10, step=0.1, value=5.0, 
                           tooltip={'always_visible':True},updatemode='drag')
slider_nu     = dcc.Slider(id='nu', min=1e-5, max=10, step=0.1, value=1.0, 
                           tooltip={'always_visible':True},updatemode='drag')
slider_mu1    = dcc.Slider(id='mu1', min=1e-5, max=10, step=0.1, value=1.0, 
                           tooltip={'always_visible':True},updatemode='drag')
slider_sigma1 = dcc.Slider(id='sigma1', min=1e-5, max=2, step=0.01, value=1.0, 
                           tooltip={'always_visible':True},updatemode='drag')
slider_mu2    = dcc.Slider(id='mu2', min=1e-5, max=10, step=1.0, value=1.0, 
                           tooltip={'always_visible':True},updatemode='drag')
slider_sigma2 = dcc.Slider(id='sigma2', min=1e-5, max=2, step=0.01, value=1.0, 
                           tooltip={'always_visible':True},updatemode='drag')

## Layout:
app.layout = html.Div([
    dbc.Container(children=[
        dcc.Markdown(text_top, dangerously_allow_html=True), 
        html.Br(),
        dbc.Row(children=[dbc.Col(children=[slider_x_max], className="col-md-8"),
                          dbc.Col(children=["|x_max| for plotting domain"], className="col-md-4")]),
        html.Br(),
        dbc.Row(children=[dbc.Col(children=[slider_y_max], className="col-md-8"),
                          dbc.Col(children=["|y_max| for plotting domain"], className="col-md-4")]),
        html.Br(),
        dbc.Row(children=[dbc.Col(children=[slider_nu], className="col-md-8"),
                          dbc.Col(children=["nu (threshold from diagonal)"], className="col-md-4")]),
        html.Br(),
        dbc.Row(children=[dbc.Col(children=[slider_mu1], className="col-md-8"),
                          dbc.Col(children=["mu 0"], className="col-md-4")]),
        html.Br(),
        dbc.Row(children=[dbc.Col(children=[slider_sigma1], className="col-md-8"),
                          dbc.Col(children=["sigma 0"], className="col-md-4")]),
        html.Br(),
        dbc.Row(children=[dbc.Col(children=[slider_mu2], className="col-md-8"),
                          dbc.Col(children=["mu 1"], className="col-md-4")]),
        html.Br(),
        dbc.Row(children=[dbc.Col(children=[slider_sigma2], className="col-md-8"),
                          dbc.Col(children=["sigma 0"], className="col-md-4")]),
        dcc.Graph(id='plot', figure=go.Figure()),
        html.Br(),
        html.Br(),
        dcc.Markdown(text_bottom, dangerously_allow_html=True)
        
    ]),
    ###### important for latex ######
#     mathjax_script,
])


# callback - 1 (from slider)
@app.callback(Output('plot', 'figure'),
             [Input('x_max', 'value'),
              Input('y_max', 'value'),
              Input('nu', 'value'),
              Input('mu1', 'value'),
              Input('sigma1', 'value'),
              Input('mu2', 'value'),
              Input('sigma2', 'value')])

def update_plot(s1, s2, s3, s4, s5, s6, s7):
    
    xMax, yMax, nu, mu1, sigma1, mu2, sigma2 = float(s1), float(s2), float(s3), float(s4), float(s5), float(s6),float(s7)
    
    xx, yy, zz = makegrid(xMax,yMax, [mu1, mu2], [sigma1, sigma2], nu, resolution=10)
    return display_plot([xx, yy, zz])

In [ ]:
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [01/Nov/2020 10:47:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 10:47:20] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 10:47:20] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 10:47:20] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 10:47:24] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 10:47:24] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 10:47:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 10:47:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 10:47:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 10:47:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 10:47:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 10:47:25] "POST /_dash-update-component

127.0.0.1 - - [01/Nov/2020 10:47:35] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 10:47:35] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 10:47:35] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 10:47:35] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 10:47:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 10:47:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 10:47:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 10:47:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 10:47:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 10:47:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 10:47:40] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 10:47:40] "POST /_dash-update-component HTTP/1.1" 200 -
127.